We reccomend you calculate the Energy Use Intensity (kBtu/GSF) before training your model. Since the various energy uses are in different units you must first convert them to kBtu, then sum them up to get the total anunal energy use, and lastly divide by the building gross square footage. You will find some starter code in this notebook to calculate the WUI.

$$eui = \dfrac{annual\,energy\, use\,(kBtu)}{building\,square\,footage\,(GSF)}, \, where\, annual\,energy\,use = electricity(kBtu) + gas(kBtu) + oil(kBtu)$$

In [3]:
def kWh_to_kBTu(electricity):
    "Converts kilowatt-hours(kWh) to kilobtus(kBtu)"
    return electricity*3.4121


# example use case
# df['E'] = df['E'].apply(kWh_to_kBTu)

In [4]:
def therms_to_kBTu(gas):
    "Converts therms(thm) to kilobtus(kBtu)"
    return gas*99.976

# example use case
# df['G'] = df['G'].apply(therms_to_kBTu)

In [9]:
def gallons_to_kBTu(oil):
    "Converts gallons to kilobtus(kBtu)"
    return oil*139

# example use case
# df['O'] = df['O'].apply(gallons_to_kBTu) 

In [10]:
def eui(E, G, O, GSF): 
    """
    Calculates energy use intensity given energy uses and building gross square footage. 
    : param E - electricity use in kBtu
    : param G - gas use in kBtu
    : param O - natural oil use in kBtu
    : param GSF - building gross square footage
    """
    annual_energy_use = E + G + O
    return annual_energy_use/GSF
    
# example use case
# df['eui'] = df.apply(lambda: eui(df['E'], df['G'], df['O'], df['GSF']))

In [22]:
import pandas as pd

# read the building data from a CSV file
research_df = pd.read_csv('./data/research.csv')


#converting the lectricity, oil and gas to kBTu
research_df['E_kBTu'] = research_df['E'].apply(kWh_to_kBTu)
research_df['G_kBTu'] = research_df['G'].apply(therms_to_kBTu)
research_df['O_kBTu'] = research_df['O'].apply(gallons_to_kBTu)

research_df.head(5)

,Building Code,Address,Property Type,E,G,O,Building Gross Footage,E_kBTu,G_kBTu,O_kBTu
0,581,2 Cummington Mall,Research,1486320,42183,0.0,59825.75,5.071472e+06,4.217288e+06,0.0
1,582,30-38 Cummington Street,Research,1309308,43468,5531.0,36466.44,4.467490e+06,4.345757e+06,768809.0
2,587,42-44 Cummington Street,Research,3800070,106247,0.0,101335.67,1.296622e+07,1.062215e+07,0.0
3,595,3-5 Cummington Street (616-620 Comm. Ave.),Research,3957840,139459,0.0,152457.33,1.350455e+07,1.394255e+07,0.0
4,597,631-639 Commonwealth Avenue,Research,1588680,39331,190.4,100339.72,5.420735e+06,3.932156e+06,26465.6


In [32]:
# research_df['eui'] = [eui(research_df.iloc[i]['E_kBTu'],
# research_df.iloc[i]['G_kBTu'],
# research_df.iloc[i]['O_kBTu'],
# research_df.iloc[i]['Building Gross Footage']) for i in range(len(research_df))]

research_df['eui'] = research_df.apply(lambda x : eui(x['E_kBTu'], x['G_kBTu'], x['O_kBTu'], x['Building Gross Footage']),axis=1)


In [33]:
research_df.head(5)

,Building Code,Address,Property Type,E,G,O,Building Gross Footage,E_kBTu,G_kBTu,O_kBTu,eui
0,581,2 Cummington Mall,Research,1486320,42183,0.0,59825.75,5.071472e+06,4.217288e+06,0.0,155.263579
1,582,30-38 Cummington Street,Research,1309308,43468,5531.0,36466.44,4.467490e+06,4.345757e+06,768809.0,262.763670
2,587,42-44 Cummington Street,Research,3800070,106247,0.0,101335.67,1.296622e+07,1.062215e+07,0.0,232.774589
3,595,3-5 Cummington Street (616-620 Comm. Ave.),Research,3957840,139459,0.0,152457.33,1.350455e+07,1.394255e+07,0.0,180.031349
4,597,631-639 Commonwealth Avenue,Research,1588680,39331,190.4,100339.72,5.420735e+06,3.932156e+06,26465.6,93.476010
